# NOTEBOOK PARA GENERAR EL DATAFRAME CON LA DATA DE TODOS LOS ARCHIVOS PARA AAFIT, BBFIT Y OTROS

<font size=5>La data para los hits la generaré con otro Notebook.</font>

Es importante mencionar que esta "limpieza" (generación del algoritmo) y obtención de data, me llevó aproximadamente 12 horas en general, desde el proceso creativo, pasando por la frustración, muchos intentos fallidos y hasta el momento de EUREKA!!!... XD.

Borré ya la mayoría de celdas que me llevaron al código de las siguientes dos celdas, el cual está mucho más pulido y limpio, pero creo que es importente mencionar por lo que pase y las ideas (algunas locas) que me surgieron para poder resolverlo.

Algunos ideas que tuve antes de llegar a decidir que no calcularia como tal el indice de aafit fueron:
* Solo quedarme con los archivos que tuvieran 100% data completa (terrible idea ya que son más del 50% los archivos que en algunos eventos - alrededor del 5% o menos - no tienen data de aafit).
* Verificar las longitudes de la cadena aafit y bbfit y comparar (idea bastante mala por cierto)
* Tratar de buscar por separado en otra celda solo la data de aafit y "pegarla" de alguna manera al dataframe final, hay que decir que esta idea también fué muy mala y no prosperó en lo absoluto (creo no tenia ni pies ni cabeza, momento de frustración máxima).

In [1]:
#Recordando algunas variables importantes, funciones y librerías que se requerirán
import os
import pandas as pd
import re
import math

#extractpath = "U:\MASTER INT. ARTIFICIAL\TFM_DATA\TXTs" #Este path es para mi laptop en casa
#extractpath = "F:\\DATA_TFM\\TXTs" #Este path es para mi PC en el trabajo
extractpath = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\TXTs" #Este es para mi no tan nueva PC-Desktop

txtfilesarr = os.listdir(extractpath); txtfilesarr=sorted(txtfilesarr)
#csv_path = "F:\\DATA_TFM\\CSVs\\"
csv_path = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\\"
csvarr = os.listdir(csv_path);csvarr=sorted(csvarr)

# Funcion para obtener los indices de un archivo
def indices(df):
    ind_startev = [];ind_runid = [];ind_weights = [];ind_muon = []
    ind_bbfit = [];ind_bb_selp = [];ind_endev = [];#ind_aafit = []
    for i in range(data.shape[0]):
        tempstr = data[0][i]
        if "start_event" in tempstr: ind_startev.append(i)
        if "UTC" in tempstr: ind_runid.append(i)
        if "weights" in tempstr: ind_weights.append(i)
        if "muon" in tempstr: ind_muon.append(i)
        #if "aafit" in tempstr:  ##NO LO CALCULARE!!! OBTENDRE LOS VALORES DE OTRA FORMA
        #    ind_aafit.append(i) ##COMO TENGO EL INDICE DE BBFIT, RESTO 1 LINEA Y ASIGNO VALORES
        if "bbfit" in tempstr:   ##SEGUN LA EXISTENCIA O NO DE LA LINEA!!!! EUREKA!!!
            ind_bbfit.append(i)
        if "selected" in tempstr: ind_bb_selp.append(i)
        if "end_event" in tempstr: ind_endev.append(i)
    return(ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev)

#Función para verificar si los indices tienen el mismo # de elementos
def longitudes(i1,i2,i3,i4,i5,i6,i7):
    a=len(i1);b=len(i2);c=len(i3);d=len(i4);e=len(i5);f=len(i6);g=len(i7)
    #print(a,b,c,d,e,f,g)
    if (a==b and b==c and c==d and d==e and e==f and f==g): return(True)
    else: return(False)
    return()

### NOTA: Como siempre, la gran cuestión y prerrogativa humana es encontrar patrones en los datos y automatizar
### búsquedas aún y cuando estos no se vean a simple vista o no sean fácilmente inferibles, tales como cuando 
### el patrón es precisamente eso... que NO HAY PATRÓN!!!, esto aplica para el no indice ind_aafit

In [7]:
############################################################################################################
###### ESTE CODIGO GENERARÁ UN DATAFRAME CON LA DATA DE BBFIT, AAFIT Y OTRAS DE ""TODOS"" LOS ARCHIVOS #####
############################################################################################################

#Se requiere un dataframe donde iremos guardando cada fila de data obtenida de cada archivo
columnas = ['runID','frameID','trigger_counter','interactionID','aafit_azimut',
            'aafit_zenit','aafit_lambda','aafit_beta','bbfit_azimut','bbfit_zenit','bbfit_quality',
            'muon_azimut','muon_zenit','muon_energy'] #La data del muon la añadi el 30 Jun, lo habia "olvidado" XD
df_data1 = pd.DataFrame(columns=columnas)

consecutivo=0 #variable auxiliar para guardar el indice del ultimo elemento del archivo

# Bucle para recorrer TODOS los archivos y obtener la data y metadata de TODOS
for k in range(25): #len(csvarr))
    #print("Ciclo", k)
    file = csv_path+csvarr[k]
    print("El archivo trabajando es: ",file)
    data = pd.read_csv(file, header=None) # Dataframe del archivo en tratamiento
    ind_startev = [];ind_runid = [];ind_weights = [];ind_muon = []
    ind_bbfit = [];ind_bb_selp = [];ind_endev = []

    #Generacion de indices para cada archivo
    ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev = indices(data)

    #Como el runid es identico para todos los datos en un mismo archivo, uso el primero que aparece en el dataframe
    runID = int(list(str(data[0][ind_runid[0]]).split(" "))[0])
    #Igual como el interactionID es el mismo para todos, asigno 1 si el archivo es "numu" y 2 si es "anumu"
    interactionID = [lambda:1, lambda:2]["anumu" in str(file)]()
    # Bucle para recorrer el archivo en curso y obtener la data relevante
    print("Entrando al bucle para meter renglones al dataframe") 
    for i in range(len(ind_startev)):
        #Codigo para obtener los demas datos por cada evento y despues asignarlo a los hits
        #Divido la cadena en los valores que la componen para después extraer frameid y trigger_counter
        temprunid = list(data[0][ind_runid[i]].split(" "))
        tempmuon = list(data[0][ind_muon[i]].split(" "))
        tempbbfit = list(data[0][ind_bbfit[i]].split(" "))
        #Extrayendo las variables frameid, trigger_counter de este renglon---> esta data si cambia ojo!!
        frameID = temprunid[1]; trigger_counter = temprunid [2]
        #Obteniendo y generando Datos para aafit
        #Para calcular la data solo haremos la diferencia y si exista data la calculamos si no
        #pues ponemos valores fuera de rango para saber después que no hubo data como -3pi
        laafit = data[0][ind_bbfit[i]-1]
        if "aafit" in laafit:
            tempaafit = list(data[0][ind_bbfit[i]-1].split(" "))
            aafit_azimut = math.atan2(float(tempaafit[3]),float(tempaafit[2]))
            aafit_zenit = math.acos(float(tempaafit[4]))
            aafit_lambda = tempaafit[-2]
            aafit_beta = tempaafit [-1]
        else:
            aafit_azimut=-3*math.pi;aafit_zenit = -3*math.pi;aafit_lambda=1;aafit_beta=-1
        #Obteniendo y generando Datos para bbfit
        X = float(tempbbfit[2]); Y = float(tempbbfit[3]); Z = float(tempbbfit[4])
        #Hay que verificar si X o Y son nan y asignar el valor de bbfit_azimut
        if math.isnan(X):   bbfit_azimut = -3*math.pi
        elif math.isnan(Y): bbfit_azimut = -3*math.pi
        else: bbfit_azimut = math.atan2(Y,X)
        bbfit_zenit = math.acos(Z)
        bbfit_quality = float(tempbbfit[-1]) 
        #Obteniendo y generando la data del muon
        X = float(tempmuon[1]); Y = float(tempmuon[2]); Z = float(tempmuon[3])
        muon_azimut = math.atan2(Y,X)
        muon_zenit = math.acos(Z)
        muon_energy = float(tempmuon[-2])
        #Añadimos la data obtenida y la generada al dataframe
        renglon = [runID, frameID, trigger_counter, interactionID,aafit_azimut, aafit_zenit,
                   aafit_lambda,aafit_beta, bbfit_azimut,bbfit_zenit,bbfit_quality,muon_azimut,muon_zenit,muon_energy]
        #Añadiendo datos al dataframe.
        df_data1.loc[i+consecutivo] = renglon
    consecutivo = df_data1.shape[0]

    #consecutivo=i+1 #Sumar 1 es muy importante si no, sobreescribe el ultimo valor del evento, del archivo respectivo
print("Fin de la extracción de la data")

El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025800_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025800_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025880_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025880_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025920_anumu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025920_numu_CC_a_reco.i3.csv
Entrando al bucle para meter renglones al dataframe
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025930_anumu_CC_a_rec

In [14]:
### PRUEBAS... BORRAR ESTA CELDA AL FINAL

#Viendo como quedó el dataframe
print("El número de filas del dataframe es:", df_data1.shape[0])

print("Una muestra de las filas y la data en ellas")

df_data1.sample(10)

#df_data1.iloc[0:250]

El número de filas del dataframe es: 5155
Una muestra de las filas y la data en ellas


,runID,frameID,trigger_counter,interactionID,aafit_azimut,aafit_zenit,aafit_lambda,aafit_beta,bbfit_azimut,bbfit_zenit,bbfit_quality,muon_azimut,muon_zenit,muon_energy
2076,25990,110171,19,1,0.008170,0.423677,-6.45441076961,0.0154777204448,-9.424778,0.412248,1.313463,0.026999,0.675214,24.8090
1114,25920,9979,143,2,-2.926711,0.839339,-6.34336377532,0.123222929322,2.531700,1.229617,2.934493,-2.463634,0.349742,333.7690
3562,26190,140849,275,2,1.381038,1.010685,-5.90871984937,0.0261208876628,2.246594,1.483662,1.865748,2.011759,1.499025,7039.0700
1699,25930,97351,172,2,0.911531,0.714967,-5.58818350355,0.00798197815014,-9.424778,0.656172,0.960881,1.071556,0.725642,7939.4800
1094,25920,78911,123,2,2.337692,1.159702,-6.2470896742,0.0407885708822,-9.424778,1.184103,6.608104,2.625332,1.755859,207.8210
986,25920,125919,6,2,-1.382817,0.356283,-5.50046034996,0.0136745911615,-9.424778,0.256969,0.645198,-0.352943,0.291665,19.8327
5000,26340,30781,125,2,2.316448,1.273985,-5.52610126548,0.0108518854219,2.323447,1.246598,1.186672,2.305526,1.269835,215.0780
1933,25990,53483,21,2,-2.455953,0.406044,-5.31446152589,0.00926182047559,-2.382000,0.393877,0.553995,-1.988018,0.397162,102.2160
4934,26340,23365,57,2,-1.606926,0.690241,-4.8832394884,0.00826000187661,-9.424778,0.619391,1.577757,-1.613415,0.698633,57.1997
2456,26030,51043,20,2,-9.424778,-9.424778,1,-1,-0.685443,1.499144,4.044593,-2.203849,2.262202,9111.8300


In [16]:
## Una vez que tenemos el dataframe con toda la data relevante de aafit y bbfit
## debemos guardar esta data a disco

outputpath = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CLEANDATA\\"
    
df_data1.to_csv (outputpath+"data_aafit_bbfit_muon"+".csv", index = None)#, header=None)

# EL SIGUIENTE CÓDIGO NO ES NECESARIO PERO LO DEJÉ COMO EVIDENCIA DE LO QUE HABÍA ESTADO HACIENDO INICIALMENTE

In [ ]:
##########################################################################################
####  ESTE CODIGO CONTIENE EL PRIMER ALGORITMO QUE GENERÉ PARA OBTENER LA DATA DE      ###
####  UN ARCHIVO CSV YA CON TRATAMIENTO "INICIAL", ES DECIR SIN LINEAS VACIAS Y OTRAS  ### 
####          ESTABA DIVIDIDO EN DOS CELDAS POR TODO EL TESTING QUE HICE               ###
##########################################################################################

    ####  #####  #####  #####  #####
    ##    ####   ####   #   #  ####
    ####  #   #  #   #  #####  #   #
    
# -->ESTE ALGORITMO TUVO UN FALLO FUNDAMENTAL... NO TOMABA EN CUENTA QUE HAY ARCHIVOS QUE NO TIENEN DATA DE AAFIT
# -->Sin embargo me sirvió como base primaria para generar el adecuado (2da. celda de arriba a abajo de este notebook)

file = csv_path+csvarr[1]
print("El archivo trabajando es: ",file)
data = pd.read_csv(file, header=None) # Dataframe del archivo en tratamiento
ind_startev = [];ind_runid = [];ind_weights = [];ind_muon = []
ind_aafit = [];ind_bbfit = [];ind_bb_selp = [];ind_endev = []

for i in range(data.shape[0]):
    tempstr = data[0][i]
    if "start_event" in tempstr:
        ind_startev.append(i)
    if "UTC" in tempstr:
        ind_runid.append(i)
    if "weights" in tempstr:
        ind_weights.append(i)
    if "muon" in tempstr:
        ind_muon.append(i)
    if "aafit" in tempstr:
        ind_aafit.append(i)
    if "bbfit" in tempstr:
        ind_bbfit.append(i)
    if "selected" in tempstr:
        ind_bb_selp.append(i)
    if "end_event" in tempstr:
        ind_endev.append(i)

####### YA QUE LOGRE GENERAR EL DATAFRAME PARA LOS DATOS DE AAFIT Y BBBFIT DE UN SOLO EVENTO,
####### AHORA LO HARÉ PARA TODOS LOS EVENTOS EN EL MISMO ARCHIVO
####### CODIGO PARA OBTENER LA DATA AAFIT, BBFIT, TRANSFORMACIONES Y OTROS PARA EL DATAFRAME

#Como el runid es identico para todos los datos en un mismo archivo, uso el primero que aparece en el dataframe
runID = int(list(str(data[0][ind_runid[0]]).split(" "))[0])

#Igual como el interactionID es el mismo para todos, asigno 1 si el archivo es "numu" y 2 si es "anumu"
if "anumu" in str(file):
    interactionID = 2
else:
    interactionID = 1
    
#Se requiere un dataframe donde iremos guardando cada fila de data obtenida
columnas = ['runID','frameID','trigger_counter','interactionID','aafit_azimut','aafit_zenit',
            'aafit_lambda','aafit_beta','bbfit_azimut','bbfit_zenit','bbfit_quality']
df_data1 = pd.DataFrame(columns=columnas)

#Ahora generamos un bucle para recorrer todo el archivo y usamos los indices
for i in range(len(ind_startev)):
    #Codigo para obtener los demas datos por cada evento y despues asignarlo a los hits
    #Divido la cadena en los valores que la componen para después extraer frameid y trigger_counter
    temprunid = list(data[0][ind_runid[i]].split(" "))
    tempmuon = list(data[0][ind_muon[i]].split(" "))
    tempaafit = list(data[0][ind_aafit[i]].split(" "))
    tempbbfit = list(data[0][ind_bbfit[i]].split(" "))
    #Extrayendo las variables frameid, trigger_counter de este renglon---> esta data si cambia ojo!!
    frameID = temprunid[1]; trigger_counter = temprunid [2]
    #Obteniendo y generando Datos para aafit
    aafit_azimut = math.atan2(float(tempaafit[3]),float(tempaafit[2]))
    aafit_zenit = math.acos(float(tempaafit[4]))
    aafit_lambda = tempaafit[-2]
    aafit_beta = tempaafit [-1]
    #Obteniendo y generando Datos para bbfit
    X = float(tempbbfit[2]); Y = float(tempbbfit[3]); Z = float(tempbbfit[4])
    #Hay que verificar si X o Y son nan y asignar el valor de bbfit_azimut
    if math.isnan(X):   bbfit_azimut = -3*math.pi
    elif math.isnan(Y): bbfit_azimut = -3*math.pi
    else: bbfit_azimut = math.atan2(Y,X)
    bbfit_zenit = math.acos(Z)
    bbfit_quality = float(tempbbfit[-1]) 
    #Añadimos la data obtenida y la generada al dataframe
    renglon = [runID, frameID, trigger_counter, interactionID,aafit_azimut, aafit_zenit,
               aafit_lambda,aafit_beta, bbfit_azimut,bbfit_zenit,bbfit_quality]
    df_data1.loc[i] = renglon
    
